# Introduction

Plonk is a tool for analysis and visualisation of smoothed particle hydrodynamics (SPH) data. To use Plonk we import it directly. You can see what classes and functions are available with the `help` command.

In [ ]:
import plonk
help(plonk)

The following sets the data directory and file prefix for convenience in this notebook

In [ ]:
import pathlib
DIRECTORY = pathlib.Path('~/runs/twhya/2018-03-09b').expanduser()
PREFIX = 'twhya'

## Dump object

SPH dump files are represented by the `Dump` class. This object contains a header dictionary, particle arrays, which are represented by an `Arrays` class. Here we demonstrate instantiating a `Dump` object, and accessing the header and particle arrays.

In [ ]:
filename = DIRECTORY / (PREFIX + '_00250.h5')
dump = plonk.Dump(filename)

We can access the dump header as a dictionary.

In [ ]:
dump.header

The particle arrays are stored in the HDF5 file and accessed as required. I.e. we don't read all the data into memory, although you can if you wish by using the `cache_arrays` method on the `Dump` object.

In [ ]:
dump.particles

This returns an `Arrays` object which stores particle arrays, and has methods to access the data. For example, see what arrays are stored in the dump file.

In [ ]:
dump.particles.fields

To access the arrays we use the `arrays` dictionary which contains h5py pointers to the data on disc.

In [ ]:
dump.particles.arrays['xyz'][:]

If the dump contains sink particles, we can access them in the same way.

In [ ]:
dump.sinks.fields

## Evolution object

SPH simulations usually output data other than dump files. For example, Phantom outputs `.ev` files which contain global data, such as kinetic energy or angular momentum. The data is written more frequently that dump files but the quantity of data is reduced. As such these files are text files.

Plonk has the `Evolution` class to represent this data. Here we instantiate an `Evolution` object from either a single file, or a collection of files organised in chronological order.

In [ ]:
# Single file Evolution object
evfile = DIRECTORY / (PREFIX + '01.ev')
print(evfile.name)
evol = plonk.Evolution(evfile)

In [ ]:
# Multiple file Evolution object
evfiles = [DIRECTORY / (PREFIX + f'{index:02}.ev') for index in [1, 2, 3]]
for evfile in evfiles:
    print(evfile.name)
evol = plonk.Evolution(evfiles)

The data is represented by a Pandas `DataFrame`, so we can use the built-in methods from Pandas, such as plotting.

In [ ]:
# Accessing the data.
evol.data['time'], evol.data['etherm']

In [ ]:
# Plotting quantities against time.
evol.plot('xcom', 'ycom')

## Simulation object

We see that for a single SPH simulation the data is spread over multiple files of multiple types, even though, logically, a simulation is a singular "object". In Plonk we have the `Simulation` class to represent the totality of the simulation data. It is an aggregation of the `Dump` and `Evolution` objects, plus metadata, such as the directory on the file system.

In [ ]:
sim = plonk.Simulation(prefix=PREFIX, directory=DIRECTORY)

# Accessing the Dump objects.
sim.dumps

# Visualisation

## Visualization object

The `Visualization` class provides methods to visualise SPH data. Plonk uses Splash for interpolation to a pixel array. Instantiation of a `Visualization` object produces a figure.

The following examples produce a rendered image. Each example demonstrates a different method for the user to choose the quantity they wish to render.

The first quantity is a string representing the name of the quantity to render. Any scalar dump particle array quantity can be rendered, as well as several predefined extra quantities.


In [ ]:
viz = plonk.Visualization(
    dump=dump,
    render='density',
    size=250,
)

The second example shows an extra quantity to be calculated. We use SymPy to parse the string.

In [ ]:
viz = plonk.Visualization(
    dump=dump,
    render='sqrt(vx**2 + vy**2)',
    size=250,
)

In the final example we show that you can calculate any quantity from the underlying data using NumPy array operations.

In [ ]:
# Calculate the deviation from Keplerian velocity.
vx = dump.particles.arrays['vxyz'][:, 0]
vy = dump.particles.arrays['vxyz'][:, 1]
G = plonk.constants.gravitational_constant / (
    dump.header['udist'] ** 3
    / dump.header['umass']
    / dump.header['utime'] ** 2
)
M = dump.sinks.arrays['m'][0]
R = dump.extra_quantity('R')[0]
deviation_from_keplerian = np.sqrt(vx ** 2 + vy ** 2) - np.sqrt(G * M / R)

In [ ]:
# Focus on planet location
WINDOW_SIZE = 100
PLANET_INDEX = 3

planet_x = dump.sinks.arrays['xyz'][PLANET_INDEX, 0]
planet_y = dump.sinks.arrays['xyz'][PLANET_INDEX, 1]
extent = [
    planet_x - WINDOW_SIZE / 2,
    planet_x + WINDOW_SIZE / 2,
    planet_y - WINDOW_SIZE / 2,
    planet_y + WINDOW_SIZE / 2,
]

# Make figure
viz = plonk.Visualization(
    dump=dump,
    render=deviation_from_keplerian,
    cross_section=True,
    extent=extent,
    colormap='RdBu',
)

We can also visualise vector quantities

In [ ]:
viz = plonk.Visualization(
    dump=dump,
    vector='velocity',
    size=250,
)

## Visualization methods

Once we have the `Visualization` object we can manipulate it. For example, we can rotate the frame around an arbitrary vector.

In [ ]:
viz = plonk.Visualization(
    dump=dump,
    render='density',
    size=250,
)

# Rotate frame around arbitrary vector.
viz.rotate_frame(
    axis=[1, 1, 0],
    angle=np.pi/3
)

Or change the window size and the particle type, and so on.

In [ ]:
viz = plonk.Visualization(
    dump=dump,
    render='density',
    size=250,
)

# Set particle type.
I_DUST = 2
viz.set_particle_type(I_DUST)
viz.set_image_size(size=90)

## MultiPlot

TODO

## VisualizationIterator

TODO

# Analysis

Plonk can perform analysis on the SPH data. The function `analysis.disc` is equivalent to the Phantom analysis module available in `analysis_disc.f90`, which computes azimuthally-averaged quantities on an accretion disc.

This analysis assumes a single disc around a single star (represented as a sink particle). We need to define the number of radial bins to average our data, as well as the inner and outer disc radius.

In [ ]:
# Radially bin disc quantities.
av = plonk.analysis.disc(
    dump=dump,
    radius_in=10,
    radius_out=200
)

The analysis produces Pandas `DataFrames` with index associated with the radial bin. We can plot the data using built-in methods, or with Matplotlib.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

ax[0].plot(av['R'], av['sigma'])
ax[0].set_xlabel('Radius')
ax[0].set_ylabel('Surface density')

ax[1].plot(av['R'], av['H'])
ax[1].set_xlabel('Radius')
ax[1].set_ylabel('Scale height')

# TODO

+ units
+ where to get data set
+ multiplot
+ viz iteration